In [1]:
!pip install delta-spark==1.2.0

In [2]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [3]:
print(pyspark.__version__)

3.2.1


In [4]:
conf = pyspark.SparkConf().set("spark.jars.packages","org.mongodb.spark:mongo-spark-connector_2.12:3.0.1").setMaster("local").setAppName("Spark-Job").setAll([('spark.driver.memory', '1g'),('spark.executor.memory','1g')])
sc = SparkContext(conf=conf)
sqlC = SQLContext(sc)

/usr/local/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("DeltaApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 

spark = configure_spark_with_delta_pip(builder).master("spark://spark:7077").getOrCreate()

spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://minio:9000")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "FFD34B2AC56E76E8BB9E7EFD7D283")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "47A13EFAB1CA1F253FD6F56DEF769")
spark._jsc.hadoopConfiguration().set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
spark._jsc.hadoopConfiguration().set("spark.mongodb.input.uri", "mongodb://mongodb/auto-mpg.auto")
spark._jsc.hadoopConfiguration().set("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.10-2.2.10")

In [6]:
df_business = spark.read \
        .format("json") \
        .option("inferSchema", "true") \
        .json("s3a://landing/manufacturers//*.json")

In [7]:
df_business.show()

+-------+--------------------+-------------+
|country|dt_current_timestamp| manufacturer|
+-------+--------------------+-------------+
|     us|       1670238236187|          amc|
|germany|       1670238236187|         audi|
|germany|       1670238236187|          bmw|
|     us|       1670238236187|        buick|
|     us|       1670238236187|     cadillac|
|     us|       1670238236187|        capri|
|     us|       1670238236187|    chevroelt|
|     us|       1670238236187|    chevrolet|
|     us|       1670238236187|        chevy|
|     us|       1670238236187|     chrysler|
|  japan|       1670238236187|       datsun|
|     us|       1670238236187|        dodge|
|  italy|       1670238236187|         fiat|
|     us|       1670238236187|         ford|
|     us|       1670238236187|           hi|
|  japan|       1670238236187|        honda|
|  japan|       1670238236187|        maxda|
|  japan|       1670238236187|        mazda|
|germany|       1670238236187|     mercedes|
|germany| 

In [8]:
# MongoDB Details
mongo_ip = "mongodb://mongodb:27017/auto-mpg."

In [9]:
print(mongo_ip)

mongodb://mongodb:27017/auto-mpg.


In [10]:
iris = sqlC.read.format('com.mongodb.spark.sql.DefaultSource').option("uri", mongo_ip + "auto").load()
iris.createOrReplaceTempView("auto")
iris= sqlC.sql("SELECT * FROM auto")
iris.show()

+--------------------+------------+--------------------+---------+------------+----------+-----+---------+----+------+------+
|                 _id|acceleration|             carname|cylinders|displacement|horsepower|index|modelyear| mpg|origin|weight|
+--------------------+------------+--------------------+---------+------------+----------+-----+---------+----+------+------+
|{638ba5251833991e...|        12.0|chevrolet chevell...|        8|       307.0|     130.0|    0|       70|18.0|     1|3504.0|
|{638ba5251833991e...|        11.5|   buick skylark 320|        8|       350.0|     165.0|    1|       70|15.0|     1|3693.0|
|{638ba5251833991e...|        11.0|  plymouth satellite|        8|       318.0|     150.0|    2|       70|18.0|     1|3436.0|
|{638ba5251833991e...|        12.0|       amc rebel sst|        8|       304.0|     150.0|    3|       70|16.0|     1|3433.0|
|{638ba5251833991e...|        10.5|         ford torino|        8|       302.0|     140.0|    4|       70|17.0|     1|